In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits import mplot3d
import matplotlib.animation as animation

In [ ]:
def Func(x1, x2):
    r = np.sqrt((x1 - np.pi)**2 + (x2 - np.pi)**2)
    if r==0:
        return 0
    elif r > 0 and r < np.pi/2:
        return 10*np.exp(-1/r)*np.exp(-1/((np.pi/2) - r))
    else:
        return 0

def gamma(x1, x2):
    return 0.01*(1+Func(x1,x2))

def phi0(x1, x2):
    return np.sin(2*x1)**2 + x2*0

In [ ]:
dx = np.pi/64
dy = np.pi/64
x1 = np.arange(0, 2*np.pi, dx)
x2 = np.arange(0, 2*np.pi, dy)
nx = len(x1)
ny = len(x2)
X1, X2 = np.meshgrid(x1, x2)
print(nx)
print(ny)

In [ ]:
u1 = np.zeros((nx,ny))
u2 = np.zeros((nx,ny))
for i in range(nx):
    for j in range(ny):
        u1[i,j] = -(x2[j]-np.pi)*Func(x1[i],x2[j])
        u2[i,j] = (x1[i]-np.pi)*Func(x1[i],x2[j])
phi_0 = phi0(X1, X2)
phi = phi_0
dPhi = np.zeros((nx,ny,2))

In [ ]:
# fig = plt.figure()
# ax = plt.axes(projection='3d')
# ax.contour3D(X1, X2, phi, 50, cmap='binary')
# ax.set_xlabel('x1')
# ax.set_ylabel('x2')
# ax.set_zlabel('phi')
# ax.set_title('3D contour')
# plt.show()


In [ ]:
def dPhi_func(phi):
    
    dPhi = np.zeros((nx,ny,2))
    for i in range(2,ny-2):
        for j in range(0,nx-2):
            dPhi[i,j,1] = ((-1/3)*(phi[i+2,j]-phi[i-2,j])/(4*dy)) + ((4/3)*(phi[i+1,j]-phi[i-1,j])/(2*dy)) # again, x2
            dPhi[i,j,0] = ((-1/3)*(phi[i,j+2]-phi[i,j-2])/(4*dx)) + ((4/3)*(phi[i,j+1]-phi[i,j-1])/(2*dx)) # and x1 here 
    
    
    # periodic boundary conditions
    # dPhi[:,0,0] = automtically handled by python
    # dPhi[:,1,0] = automatically handled by python
    dPhi[:, -1, 0] = ((-1/3)*(phi[:,1]-phi[:,-3])/(4*dx)) + ((4/3)*(phi[:,0]-phi[:,-2])/(2*dx))
    dPhi[:, -2, 0] = ((-1/3)*(phi[:,0]-phi[:,-4])/(4*dx)) + ((4/3)*(phi[:,-1]-phi[:,-3])/(2*dx))
    
    # asymmetric derivatives for dirichlet boundary conditions
    # at 2 and ny-2 
    dPhi[1,:,1] = (-phi[4,:] + 6*phi[3,:] + 18*phi[2,:] + 10*phi[1,:] - 33*phi[0,:])/(60*dy)
    dPhi[-2,:,1] = (-phi[-5,:] + 6*phi[-4,:] + 18*phi[-3,:] + 10*phi[-2,:] - 33*phi[-1,:])/(60*dy)
    # at 1 and ny-1 
    dPhi[0,:,1] = 0
    dPhi[-1,:,1] = 0

    return dPhi

In [ ]:
def d2_Phi_func(dPhi, gamma):

    dPhi_Gamma = np.zeros((nx, ny, 2))
    for i in range(nx):
        for j in range(ny):
            dPhi_Gamma[i,j,0] = gamma(x1[i], x2[j])*dPhi[i,j,0]    #dPhi_gamma_x1
            dPhi_Gamma[i,j,1] = gamma(x1[i], x2[j])*dPhi[i,j,1]    #dPhi_gamma_x2

    d2_Phi = np.zeros((nx, ny, 2))
    #dPhi_Gamma = np.transpose(dPhi_Gamma, [1,0,2])
    for i in range(2,ny-2):
        for j in range(0,nx-2):
            d2_Phi[i,j,1] = ((-1/3)*(dPhi_Gamma[i+2,j,1]-dPhi_Gamma[i-2,j,1])/(4*dx)) + ((4/3)*(dPhi_Gamma[i+1,j,1]-dPhi_Gamma[i-1,j,1])/(2*dx))
            d2_Phi[i,j,0] = ((-1/3)*(dPhi_Gamma[i,j+2,0]-dPhi_Gamma[i,j-2,0])/(4*dy)) + ((4/3)*(dPhi_Gamma[i,j+1,0]-dPhi_Gamma[i,j-1,0])/(2*dy))

    
    # periodic boundary conditions
    # dPhi[:,0,0] = automtically handled by python
    # dPhi[:,1,0] = automatically handled by python
    d2_Phi[:, -1, 0] = ((-1/3)*(dPhi_Gamma[:,1,0]-dPhi_Gamma[:,-3,0])/(4*dx)) + ((4/3)*(dPhi_Gamma[:,0,0]-dPhi_Gamma[:,-2,0])/(2*dx))
    d2_Phi[:, -2, 0] = ((-1/3)*(dPhi_Gamma[:,0,0]-dPhi_Gamma[:,-4,0])/(4*dx)) + ((4/3)*(dPhi_Gamma[:,-1,0]-dPhi_Gamma[:,-3,0])/(2*dx))
    
    # asymmetric second derivatives for dirichlet boundary conditions
    # at 2 and ny-2 
    d2_Phi[1,:,1] = (-dPhi_Gamma[4,:,1] + 6*dPhi_Gamma[3,:,1] + 18*dPhi_Gamma[2,:,1] + 10*dPhi_Gamma[1,:,1] - 33*dPhi_Gamma[0,:,1])/(60*dy)
    d2_Phi[-2,:,1] = (-dPhi_Gamma[-5,:,1] + 6*dPhi_Gamma[-4,:,1] + 18*dPhi_Gamma[-3,:,1] + 10*dPhi_Gamma[-2,:,1] - 33*dPhi_Gamma[-1,:,1])/(60*dy)
    # at 1 and ny-1 
    d2_Phi[0,:,1] = 0
    d2_Phi[-1,:,1] = 0
    

            
    return d2_Phi


In [ ]:
dt = .01
t_final = 1
nt = int(t_final/dt)
print(nt)

In [ ]:
phi_array = []
dphi_x1_array = []
dphi_x2_array = []
d2phi_x1_array = []
d2phi_x2_array = []

for i in range(nt):
    
    dPhi = dPhi_func(phi)
    d2_Phi = d2_Phi_func(dPhi, gamma)
    phi = phi + dt*(-u1.T*dPhi[:,:,0] - u2.T*dPhi[:,:,1] + d2_Phi[:,:,0] + d2_Phi[:,:,1])
    
    
    phi_array.append(phi)
    dphi_x1_array.append(dPhi[:,:,0])
    dphi_x2_array.append(dPhi[:,:,1])
    d2phi_x1_array.append(d2_Phi[:,:,0])
    d2phi_x2_array.append(d2_Phi[:,:,1])
    
    if i%50==0:
        print(f'timestep: {i}/{nt}')

In [ ]:
# fig = plt.figure()
# ax = plt.axes(projection='3d')
# ax.contour3D(X1, X2, phi, 50, cmap='binary')
# ax.set_xlabel('x1')
# ax.set_ylabel('x2')
# ax.set_zlabel('phi')
# plt.title("phi at t=1")
# plt.show()

In [ ]:
phi[round(np.pi*5/(4*dx)),round(np.pi/dx)]


In [ ]:
fps = int(nt/5)
nSeconds = 5*t_final
snapshots1 = phi_array
print(np.shape(snapshots1))
fig = plt.figure( figsize=(3,3) )

a = snapshots1[0]
im = plt.imshow(a, interpolation='nearest', aspect='auto', origin='lower', extent=[0,2*np.pi,0,2*np.pi])
fig.colorbar(im)
def animate_func(i):
    if i % fps == 0:
        print( '.', end ='' )

    im.set_array(snapshots1[i])
    return [im]

anim = animation.FuncAnimation(
                               fig, 
                               animate_func, 
                               frames = nSeconds * fps,
                               interval = 1000 / fps, # in ms
                               )

plt.xlabel('x1')
plt.ylabel('x2')
plt.title('CDS x2Dirichlet')
#anim.save('Question2_Visuals/dt_cds_dirichlet128.mp4')

In [ ]:
# %matplotlib inline

# plt.imshow(phi, interpolation='nearest', origin='lower', extent=[0,2*np.pi,0,2*np.pi])
# plt.scatter(np.pi, np.pi*5/4, color='r', label='$(\pi, \pi*5/4)$')
# plt.title("CDS vertical Dirichlet t=1")
# plt.xlabel('x1')
# plt.ylabel('x2')
# plt.legend()
# plt.colorbar()
# plt.savefig("Question2_Visuals/t1_dirichlet.png")
# plt.show()
